In [3]:
import os
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer

from langchain_community.document_loaders import PyPDFLoader # 1.로드
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter # 2.청크
from langchain.embeddings import HuggingFaceEmbeddings # 3. 임베딩 모델
from langchain.vectorstores import FAISS # 3. 벡터 저장
from langchain.retrievers import BM25Retriever, EnsembleRetriever # 4. 검색 기법
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate

from transformers import pipeline

from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# 1. 문서 로드 및 청크 분할
loader = PyPDFLoader("한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관_20220601_(2).pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=30
)
chunks = text_splitter.split_documents(documents)
chunks = chunks[11:]  # 불필요한 청크 제외 (예: 목차)

# 각 청크에 ID 부여
for i, chunk in enumerate(chunks):
    chunk.metadata['doc_id'] = i

# 2. HuggingFace 임베딩 및 FAISS 생성
embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = FAISS.from_documents(documents=chunks, embedding=embedding)

# 3. 키워드 검색을 위한 BM25 생성
bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 3

# 4. 의미 검색 + 키워드 검색 (앙상블)
top_k = 3
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": top_k})
ensemble_retrievers = [retriever, bm25_retriever]
ensemble_retriever = EnsembleRetriever(retrievers=ensemble_retrievers, weights=[0.7, 0.3])

# 5. LLM 초기화 및 RAG 체인 구성
llm = ChatOllama(model="llama3.1", temperature=0.8, num_predict=300)

retrieval_qa_chat_prompt = ChatPromptTemplate.from_template("""
다음 컨텍스트를 바탕으로 질문에 답변해주세요. 컨텍스트에 관련 정보가 없다면,
"주어진 정보로는 답변할 수 없습니다."라고 말씀해 주세요.

컨텍스트: {context}

질문: {input}

답변:
""")

combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
rag_chain = create_retrieval_chain(ensemble_retriever, combine_docs_chain)

In [7]:
# 체인 생성
combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

# 체인 실행
query = "보험금 지급이 제한될 수 있는 경우는 뭐가 있어?"
response = rag_chain.invoke({"input": query})
print(response)

{'input': '보험금 지급이 제한될 수 있는 경우는 뭐가 있어?', 'context': [Document(metadata={'source': '한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관_20220601_(2).pdf', 'page': 473, 'doc_id': 1678}, page_content='을 지급합니다 .  \n【할증위험률】  \n피보험자의  건강상태가  회사가  정한 기준에  적합하지  않은 경우 일반위험률보다  높게 적용되\n는 위험률  \n \n2. 보험금감액법   \n계약일로부터  회사가  정하는  삭감기간 (削減期間 ) 내에 피보험자에게  재해 이외의  원인으로  해당계\n약의 삭감대상  보험금  지급사유가  발생한  경우에는  해당계약의  규정에도  불구하고  계약할 때 정\n한 삭감기간  및 보험금지급비율 에 따라 다음과  같이 보험금을  지급합니다 . 삭감기간이  경과한  이'), Document(metadata={'source': '한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관_20220601_(2).pdf', 'page': 156, 'doc_id': 513}, page_content='157 / 532 ⑤ 보험수익 자와 회사가  제2-3조(보험금의  지급사유 )에 대해 합의하지  못할 때는 보험수익자와  회사가  \n함께 제3자를 정하고  그 제3자의 의견에  따를 수 있습니다 . 제3자는 의료법  제3조(의료기관 )에 규정한  \n종합병원  소속 전문의  중에서  정하며 , 보험금  지급 사유 판정에  드는 의료비용은  회사가  전액 부담합니\n다. \n \n제 2-6 조 장해지급률에  관한 세부규정  \n이 특약은  해당사항이  없습니다 . \n \n제 2-7 조 보험금을  지급하지  않는 사유 \n회사는  다음 중 어느 한 가지로  보험금  지급사유가  발생한  때에는  보험금을  지급하지  않습니다 . \n1. 피보험자가  고의로  자신을  해친 경우 \

In [11]:
# 체인 타입 확인
print(f"RAG 체인의 타입: {type(rag_chain)}")

RAG 체인의 타입: <class 'langchain_core.runnables.base.RunnableBinding'>


In [16]:
import gradio as gr

# 질문 제출 처리
def submit_question(question, chat_history):
    try:
        print(f"질문: {question}")  # 디버깅: 입력된 질문 확인

        # RAG 체인 실행
        result = rag_chain.invoke({"input": question})  # 최신 LangChain에서는 .invoke() 사용
        print(f"RAG 체인 결과: {result}")  # 디버깅: 체인 실행 결과 확인

        # 컨텍스트 텍스트 추출
        context_docs = result.get("context", [])
        context_texts = [doc.page_content for doc in context_docs]  # 텍스트 추출

        # 답변 생성
        answer = result.get("answer", "답변을 생성할 수 없습니다.")

        # 하이라이트 추가
        highlight_parts = [
            (context, question) for context in context_texts if question in context
        ]
        chat_history.append((question, answer))  # 챗봇 대화 기록 업데이트
        return chat_history, "", highlight_parts
    except Exception as e:
        print(f"오류 발생: {e}")  # 디버깅: 터미널에 오류 출력
        return chat_history, f"오류 발생: {e}", []

# Gradio UI 구성
with gr.Blocks() as iface:
    gr.Markdown("# 보험 문서 챗봇\n보험 문서에 대해 물어보면 답하는 챗봇입니다.")

    # 챗봇 UI 및 입력 필드
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="질문을 입력하세요.", label="질문 입력")
    highlighted_text = gr.HighlightedText(label="관련 문서 하이라이트", show_legend=True)

    # 버튼 구성
    with gr.Row():
        submit_btn = gr.Button("보내기")
        clear_btn = gr.Button("전체 채팅 삭제 💫")

    # 버튼 이벤트 연결
    submit_btn.click(
        submit_question, 
        inputs=[msg, chatbot], 
        outputs=[chatbot, msg, highlighted_text]
    )
    clear_btn.click(lambda: ([], ""), None, chatbot)  # 전체 대화 기록 삭제

iface.launch(share=True, debug=True)

/home/jonghai/.local/lib/python3.10/site-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7867
* Running on public URL: https://8f3831aed6b6ddca4d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


질문: 보험금 지급이 제한될 수 있는 경우는 뭐가 있어?
RAG 체인 결과: {'input': '보험금 지급이 제한될 수 있는 경우는 뭐가 있어?', 'context': [Document(metadata={'source': '한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관_20220601_(2).pdf', 'page': 473, 'doc_id': 1678}, page_content='을 지급합니다 .  \n【할증위험률】  \n피보험자의  건강상태가  회사가  정한 기준에  적합하지  않은 경우 일반위험률보다  높게 적용되\n는 위험률  \n \n2. 보험금감액법   \n계약일로부터  회사가  정하는  삭감기간 (削減期間 ) 내에 피보험자에게  재해 이외의  원인으로  해당계\n약의 삭감대상  보험금  지급사유가  발생한  경우에는  해당계약의  규정에도  불구하고  계약할 때 정\n한 삭감기간  및 보험금지급비율 에 따라 다음과  같이 보험금을  지급합니다 . 삭감기간이  경과한  이'), Document(metadata={'source': '한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관_20220601_(2).pdf', 'page': 156, 'doc_id': 513}, page_content='157 / 532 ⑤ 보험수익 자와 회사가  제2-3조(보험금의  지급사유 )에 대해 합의하지  못할 때는 보험수익자와  회사가  \n함께 제3자를 정하고  그 제3자의 의견에  따를 수 있습니다 . 제3자는 의료법  제3조(의료기관 )에 규정한  \n종합병원  소속 전문의  중에서  정하며 , 보험금  지급 사유 판정에  드는 의료비용은  회사가  전액 부담합니\n다. \n \n제 2-6 조 장해지급률에  관한 세부규정  \n이 특약은  해당사항이  없습니다 . \n \n제 2-7 조 보험금을  지급하지  않는 사유 \n회사는  다음 중 어느 한 가지로  보험금  지급사유가  발생한  때에는  보험금을 

In [ ]:
import gradio as gr
def submit_question(question, chat_history):
    try:
        print(f"질문: {question}")  # 디버깅

        # RAG 체인 실행
        result = rag_chain.invoke({"input": question})  # .invoke() 사용
        print(f"RAG 체인 결과: {result}")  # 디버깅

        # context는 Document 객체들의 리스트입니다.
        context_docs = result.get("context", [])
        answer = result.get("answer", "답변을 생성할 수 없습니다.")

        # 하이라이트 데이터를 생성할 리스트
        highlight_parts = []

        # 각 Document에서 페이지 내용과 메타데이터를 확인
        for doc in context_docs:
            page_text = doc.page_content
            source = doc.metadata.get("source", "알 수 없음")
            page = doc.metadata.get("page", "알 수 없음")

            # 답변에 해당하는 부분 하이라이트
            if answer in page_text:
                highlight_parts.append((page_text, answer))  # 답변 부분 하이라이트
            else:
                highlight_parts.append((page_text, ""))  # 하이라이트 없음

        # 대화 기록에 답변 추가
        chat_history.append((question, answer))
        return chat_history, "", highlight_parts

    except Exception as e:
        print(f"오류 발생: {e}")  # 디버깅
        import traceback
        traceback.print_exc()
        return chat_history, f"오류 발생: {e}", []

# Gradio UI 구성
with gr.Blocks() as iface:
    gr.Markdown("# 보험 문서 챗봇\n보험 문서에 대해 물어보면 답하는 챗봇입니다.")

    # 챗봇 UI 및 입력 필드
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="질문을 입력하세요.", label="질문 입력")
    highlighted_text = gr.HighlightedText(label="관련 문서 하이라이트", show_legend=True)

    # 버튼 구성
    with gr.Row():
        submit_btn = gr.Button("보내기")
        clear_btn = gr.Button("전체 채팅 삭제 💫")

    # 버튼 이벤트 연결
    submit_btn.click(
        submit_question, 
        inputs=[msg, chatbot], 
        outputs=[chatbot, msg, highlighted_text]
    )
    clear_btn.click(lambda: ([], ""), None, chatbot)  # 전체 대화 기록 삭제

iface.launch(share=True, debug=True)

/home/jonghai/.local/lib/python3.10/site-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7867
* Running on public URL: https://feccf7087f41d3e342.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


질문: 보험금 지급이 제한될 수 있는 경우는 뭐가 있어?
RAG 체인 결과: {'input': '보험금 지급이 제한될 수 있는 경우는 뭐가 있어?', 'context': [Document(metadata={'source': '한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관_20220601_(2).pdf', 'page': 473, 'doc_id': 1678}, page_content='을 지급합니다 .  \n【할증위험률】  \n피보험자의  건강상태가  회사가  정한 기준에  적합하지  않은 경우 일반위험률보다  높게 적용되\n는 위험률  \n \n2. 보험금감액법   \n계약일로부터  회사가  정하는  삭감기간 (削減期間 ) 내에 피보험자에게  재해 이외의  원인으로  해당계\n약의 삭감대상  보험금  지급사유가  발생한  경우에는  해당계약의  규정에도  불구하고  계약할 때 정\n한 삭감기간  및 보험금지급비율 에 따라 다음과  같이 보험금을  지급합니다 . 삭감기간이  경과한  이'), Document(metadata={'source': '한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관_20220601_(2).pdf', 'page': 156, 'doc_id': 513}, page_content='157 / 532 ⑤ 보험수익 자와 회사가  제2-3조(보험금의  지급사유 )에 대해 합의하지  못할 때는 보험수익자와  회사가  \n함께 제3자를 정하고  그 제3자의 의견에  따를 수 있습니다 . 제3자는 의료법  제3조(의료기관 )에 규정한  \n종합병원  소속 전문의  중에서  정하며 , 보험금  지급 사유 판정에  드는 의료비용은  회사가  전액 부담합니\n다. \n \n제 2-6 조 장해지급률에  관한 세부규정  \n이 특약은  해당사항이  없습니다 . \n \n제 2-7 조 보험금을  지급하지  않는 사유 \n회사는  다음 중 어느 한 가지로  보험금  지급사유가  발생한  때에는  보험금을 

In [14]:
import gradio as gr
# Gradio UI 구성
with gr.Blocks() as iface:
    # 제목 및 설명
    gr.Markdown("# 보험 문서 챗봇\n보험 문서에 대해 물어보면 답하는 챗봇입니다.")

    # PDF 버튼 및 챗봇 UI
    with gr.Row():
        cancer_btn = gr.Button("생명보험")
        life_btn = gr.Button("손해보험")
        car_btn = gr.Button("자동차보험")
        glossary_btn = gr.Button("용어사전")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="뭐든지 물어보세요.", label="챗 입력")
    highlighted_text = gr.HighlightedText(label="문서 하이라이트", show_legend=True)

    # 버튼 행
    with gr.Row():
        submit_btn = gr.Button("보내기")
        retry_btn = gr.Button("다시보내기 ↩")
        undo_btn = gr.Button("이전 채팅 삭제 ❌")
        clear_btn = gr.Button("전체 채팅 삭제 💫")

    # 버튼 클릭 이벤트 설정
    def update_chatbot_with_message(chat_history, sender, message):
        chat_history.append((sender, message))  # 지정된 발신자와 메시지 추가
        return chat_history

    cancer_btn.click(
        lambda chat_history: update_chatbot_with_message(chat_history, "생명보험", "생명보험 관련 정보를 물어보세요."),
        inputs=chatbot,
        outputs=chatbot
    )
    life_btn.click(
        lambda chat_history: update_chatbot_with_message(chat_history, "손해보험", "손해보험 관련 정보를 물어보세요."),
        inputs=chatbot,
        outputs=chatbot
    )
    car_btn.click(
        lambda chat_history: update_chatbot_with_message(chat_history, "자동차보험", "자동차보험 관련 정보를 물어보세요."),
        inputs=chatbot,
        outputs=chatbot
    )
    glossary_btn.click(
        lambda chat_history: update_chatbot_with_message(
            chat_history,
            "용어사전",
            (
                "용어사전에 대한 정보는 다음 링크를 참조하세요:\n"
                "- [보험 용어사전 1](https://www.hwgeneralins.com/lounge/dic/index.do)\n"
                "- [보험 용어사전 2](https://m.blog.naver.com/blogfsc/220149083103)"
            )
        ),
        inputs=chatbot,
        outputs=chatbot
    )

    # 질문 제출 처리
    # 질문 제출 처리
    def submit_question(question, chat_history):
        try:
            print(f"질문: {question}")  # 입력된 질문 확인

            # RAG 체인 실행
            print("rag_chain 호출 전")
            result = rag_chain.invoke({"input": question})  # .invoke() 호출
            print(f"rag_chain 호출 결과: {result}")

            context = result.get("context", "")
            answer = result.get("answer", "답변을 생성할 수 없습니다.")

            # 하이라이트 추가
            highlight_parts = [(context, question)] if question in context else [(context, "")]
            chat_history.append((question, answer))
            return chat_history, "", highlight_parts
        except Exception as e:
            # 구체적인 에러 메시지 출력
            import traceback
            traceback.print_exc()  # 전체 에러 스택 추적 로그 출력
            print(f"오류 발생: {e}")  # 터미널에 오류 메시지 출력
            return chat_history, "오류 발생: 자세한 내용은 서버 로그를 확인하세요.", []


    
    submit_btn.click(
    submit_question, 
    inputs=[msg, chatbot], 
    outputs=[chatbot, msg, highlighted_text]
)


iface.launch(share=True, debug=True)

/home/jonghai/.local/lib/python3.10/site-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7867
* Running on public URL: https://a23e544897b7bc2ee2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


질문: 보험금 지급이 제한될 수 있는 경우는 뭐가 있어?
rag_chain 호출 전
rag_chain 호출 결과: {'input': '보험금 지급이 제한될 수 있는 경우는 뭐가 있어?', 'context': [Document(metadata={'source': '한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관_20220601_(2).pdf', 'page': 473, 'doc_id': 1678}, page_content='을 지급합니다 .  \n【할증위험률】  \n피보험자의  건강상태가  회사가  정한 기준에  적합하지  않은 경우 일반위험률보다  높게 적용되\n는 위험률  \n \n2. 보험금감액법   \n계약일로부터  회사가  정하는  삭감기간 (削減期間 ) 내에 피보험자에게  재해 이외의  원인으로  해당계\n약의 삭감대상  보험금  지급사유가  발생한  경우에는  해당계약의  규정에도  불구하고  계약할 때 정\n한 삭감기간  및 보험금지급비율 에 따라 다음과  같이 보험금을  지급합니다 . 삭감기간이  경과한  이'), Document(metadata={'source': '한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관_20220601_(2).pdf', 'page': 156, 'doc_id': 513}, page_content='157 / 532 ⑤ 보험수익 자와 회사가  제2-3조(보험금의  지급사유 )에 대해 합의하지  못할 때는 보험수익자와  회사가  \n함께 제3자를 정하고  그 제3자의 의견에  따를 수 있습니다 . 제3자는 의료법  제3조(의료기관 )에 규정한  \n종합병원  소속 전문의  중에서  정하며 , 보험금  지급 사유 판정에  드는 의료비용은  회사가  전액 부담합니\n다. \n \n제 2-6 조 장해지급률에  관한 세부규정  \n이 특약은  해당사항이  없습니다 . \n \n제 2-7 조 보험금을  지급하지  않는 사유 \n회사는  다음 중 어느 한 가지로  보험금  지

Traceback (most recent call last):
  File "/home/jonghai/.local/lib/python3.10/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
  File "/home/jonghai/.local/lib/python3.10/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/jonghai/.local/lib/python3.10/site-packages/gradio/blocks.py", line 2028, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "/home/jonghai/.local/lib/python3.10/site-packages/gradio/blocks.py", line 1834, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
  File "/home/jonghai/.local/lib/python3.10/site-packages/gradio/components/highlighted_text.py", line 191, in postprocess
    root=[
  File "/home/jonghai/.local/lib/python3.10/site-packages/gradio/components/highlighted_text.py", line 192, in <listcomp>
    HighlightedToken(token=o[0], class_or_

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ba1b83142ed72c9132.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://f83e56335d5b0579ce.gradio.live
Killing tunnel 127.0.0.1:7862 <> https://7eedc87843b1fa4eb5.gradio.live
Killing tunnel 127.0.0.1:7863 <> https://8644b5ba8ffe4537df.gradio.live
Killing tunnel 127.0.0.1:7864 <> https://6455afb5343d4e7862.gradio.live
Killing tunnel 127.0.0.1:7865 <> https://72892aa7d822533a85.gradio.live
Killing tunnel 127.0.0.1:7866 <> https://0fe5f79a4548044007.gradio.live
Killing tunnel 127.0.0.1:7867 <> https://a23e544897b7bc2ee2.gradio.live
